In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
Key = os.getenv("GOOGLE_API_KEY")

In [4]:
client = ChatGoogleGenerativeAI(google_api_key=Key, temperature=0.6, model="gemini-pro")

In [5]:
client.invoke("Who are you?")

AIMessage(content='I am Gemini, a multimodal AI language model developed by Google. I am designed to understand and generate human language, answer questions, and provide information. I am trained on a massive dataset of text and code, and I am constantly learning and improving.')

In [6]:
from langchain.llms import GooglePalm
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    }
}

In [8]:

TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [9]:
quiz_generation_prompt = PromptTemplate(input_variables=
                                        ["text","number","subject","tone","response_json"],
                                        template=TEMPLATE)

In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"], 
                            template=TEMPLATE)

In [11]:
quiz_chain = LLMChain(
    llm=client,
    output_key="quiz",
    prompt=quiz_generation_prompt ,
    verbose=True  
)

In [12]:
quiz_chain = LLMChain(
    llm = client, 
    prompt = quiz_generation_prompt,
    output_key="quiz",verbose=True
)

In [13]:
TEMPLATE2="""
You are an expert english grammarian and writer. Ensure You not include ###line which respresent comment. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [14]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [15]:
review_chain = LLMChain(
    llm=client,
    output_key="review",
    prompt=quiz_evaluation_prompt
)

In [16]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [17]:
file_path = '../data.text'

In [18]:
file_path

'../data.text'

In [19]:
with open(file_path ,'r' ) as file:
    TEXT =file.read()


In [20]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [21]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [22]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [23]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

c:\Users\Ojas\.conda\envs\condaenv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define


> Finished chain.

> Finished chain.


In [24]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [25]:
quiz =response.get('quiz')
print(quiz)

### RESPONSE_JSON
{"1": {"mcq": "What is the main focus of biology?", "options": {"a": "The study of rocks", "b": "The study of life", "c": "The study of weather", "d": "The study of history"}, "correct": "b"}, "2": {"mcq": "Which of the following is NOT a unifying theme in biology?", "options": {"a": "Cells", "b": "Evolution", "c": "Gravity", "d": "Energy processing"}, "correct": "c"}, "3": {"mcq": "What is the approximate age of life on Earth?", "options": {"a": "1 billion years", "b": "3.7 billion years", "c": "5 billion years", "d": "10 billion years"}, "correct": "b"}, "4": {"mcq": "Which of the following is NOT a subdiscipline of biology?", "options": {"a": "Molecular biology", "b": "Ecology", "c": "Geology", "d": "Physiology"}, "correct": "c"}, "5": {"mcq": "What is the role of organisms in an ecosystem?", "options": {"a": "To produce oxygen", "b": "To cycle nutrients and energy", "c": "To regulate temperature", "d": "To create new land"}, "correct": "b"}}


In [26]:
start_idx = quiz.find('{"1":')

# Extract the JSON part from the string
json_part = quiz[start_idx:]
print(json_part)

{"1": {"mcq": "What is the main focus of biology?", "options": {"a": "The study of rocks", "b": "The study of life", "c": "The study of weather", "d": "The study of history"}, "correct": "b"}, "2": {"mcq": "Which of the following is NOT a unifying theme in biology?", "options": {"a": "Cells", "b": "Evolution", "c": "Gravity", "d": "Energy processing"}, "correct": "c"}, "3": {"mcq": "What is the approximate age of life on Earth?", "options": {"a": "1 billion years", "b": "3.7 billion years", "c": "5 billion years", "d": "10 billion years"}, "correct": "b"}, "4": {"mcq": "Which of the following is NOT a subdiscipline of biology?", "options": {"a": "Molecular biology", "b": "Ecology", "c": "Geology", "d": "Physiology"}, "correct": "c"}, "5": {"mcq": "What is the role of organisms in an ecosystem?", "options": {"a": "To produce oxygen", "b": "To cycle nutrients and energy", "c": "To regulate temperature", "d": "To create new land"}, "correct": "b"}}


In [27]:
quiz = json.loads(json_part)

# Now you can access the quiz data as a dictionary
print(quiz)

{'1': {'mcq': 'What is the main focus of biology?', 'options': {'a': 'The study of rocks', 'b': 'The study of life', 'c': 'The study of weather', 'd': 'The study of history'}, 'correct': 'b'}, '2': {'mcq': 'Which of the following is NOT a unifying theme in biology?', 'options': {'a': 'Cells', 'b': 'Evolution', 'c': 'Gravity', 'd': 'Energy processing'}, 'correct': 'c'}, '3': {'mcq': 'What is the approximate age of life on Earth?', 'options': {'a': '1 billion years', 'b': '3.7 billion years', 'c': '5 billion years', 'd': '10 billion years'}, 'correct': 'b'}, '4': {'mcq': 'Which of the following is NOT a subdiscipline of biology?', 'options': {'a': 'Molecular biology', 'b': 'Ecology', 'c': 'Geology', 'd': 'Physiology'}, 'correct': 'c'}, '5': {'mcq': 'What is the role of organisms in an ecosystem?', 'options': {'a': 'To produce oxygen', 'b': 'To cycle nutrients and energy', 'c': 'To regulate temperature', 'd': 'To create new land'}, 'correct': 'b'}}


In [28]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})
    

In [29]:
quiz_table_data

[{'MCQ': 'What is the main focus of biology?',
  'Choices': 'a: The study of rocks | b: The study of life | c: The study of weather | d: The study of history',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is NOT a unifying theme in biology?',
  'Choices': 'a: Cells | b: Evolution | c: Gravity | d: Energy processing',
  'Correct': 'c'},
 {'MCQ': 'What is the approximate age of life on Earth?',
  'Choices': 'a: 1 billion years | b: 3.7 billion years | c: 5 billion years | d: 10 billion years',
  'Correct': 'b'},
 {'MCQ': 'Which of the following is NOT a subdiscipline of biology?',
  'Choices': 'a: Molecular biology | b: Ecology | c: Geology | d: Physiology',
  'Correct': 'c'},
 {'MCQ': 'What is the role of organisms in an ecosystem?',
  'Choices': 'a: To produce oxygen | b: To cycle nutrients and energy | c: To regulate temperature | d: To create new land',
  'Correct': 'b'}]

In [30]:
quiz=pd.DataFrame(quiz_table_data)

In [31]:
quiz.to_csv("machinelearning.csv",index=False)

In [32]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'03_15_2024_12_08_07'